In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [3]:
# Define paths and actions
actions = np.array(['MoveRight', 'MoveLeft', 'ThumbsUp', 'ThumbsDown', 'Stop', 'Circle', 'FistClose', 'Victory'])
data_path = "/kaggle/input/hand-gesture-recogniton/Action_Recognition" 
no_sequences = 100
sequence_length = 60
features = 1662  # Adjust this based on the actual features per frame
num_classes = len(actions)

In [4]:
print(f"Actions: {actions}")
print(f"Number of classes: {num_classes}")
print(f"Data path: {data_path}")

Actions: ['MoveRight' 'MoveLeft' 'ThumbsUp' 'ThumbsDown' 'Stop' 'Circle'
 'FistClose' 'Victory']
Number of classes: 8
Data path: /kaggle/input/hand-gesture-recogniton/Action_Recognition


In [5]:
# Map each action to an integer
label_map = {label: num for num, label in enumerate(actions)}
print("Label map:", label_map)

Label map: {'MoveRight': 0, 'MoveLeft': 1, 'ThumbsUp': 2, 'ThumbsDown': 3, 'Stop': 4, 'Circle': 5, 'FistClose': 6, 'Victory': 7}


In [6]:
# Load data
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            frame_path = os.path.join(data_path, action, str(sequence), f"{frame_num}.npy")
            res = np.load(frame_path)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    print(f"Loaded data for action: {action}")

Loaded data for action: MoveRight
Loaded data for action: MoveLeft
Loaded data for action: ThumbsUp
Loaded data for action: ThumbsDown
Loaded data for action: Stop
Loaded data for action: Circle
Loaded data for action: FistClose
Loaded data for action: Victory


In [7]:
# Convert labels to one-hot encoding and lists to numpy arrays
labels = to_categorical(labels, num_classes=num_classes)
sequences = np.array(sequences)

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.05, random_state=42)
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")

X_train shape: (760, 60, 1662), X_test shape: (40, 60, 1662)
y_train shape: (760, 8), y_test shape: (40, 8)


In [9]:
# Build the model
model = Sequential()
model.add(GRU(64, return_sequences = True, activation = 'relu', input_shape = (60,1662)))
model.add(GRU(256, return_sequences = True, activation = 'relu'))
model.add(GRU(128, return_sequences = True, activation = 'relu'))
model.add(GRU(64, return_sequences = False, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))
print("Model has been built.")

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model has been built.


In [10]:
# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print("Model compilation complete.")

Model compilation complete.


In [11]:
# Callbacks
tb_callback = TensorBoard(log_dir='./logs', update_freq='batch')
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
print("Callbacks have been set.")

Callbacks have been set.


In [12]:
# Train the model
print("Starting model training...")
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=200, callbacks=[early_stopping, tb_callback])
print("Model training complete.")

Starting model training...
Epoch 1/200


I0000 00:00:1717259368.892676     136 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1717259368.963198     136 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


24/24 ━━━━━━━━━━━━━━━━━━━━ 21s 392ms/step - categorical_accuracy: 0.1460 - loss: 2.0809 - val_categorical_accuracy: 0.0500 - val_loss: 2.0870
Epoch 2/200
 3/24 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - categorical_accuracy: 0.1267 - loss: 2.0786

W0000 00:00:1717259378.108463     136 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - categorical_accuracy: 0.1591 - loss: 2.0784 - val_categorical_accuracy: 0.0500 - val_loss: 2.0856
Epoch 3/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - categorical_accuracy: 0.1578 - loss: 2.0782 - val_categorical_accuracy: 0.0500 - val_loss: 2.0906
Epoch 4/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - categorical_accuracy: 0.1276 - loss: 2.0782 - val_categorical_accuracy: 0.0500 - val_loss: 2.0949
Epoch 5/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - categorical_accuracy: 0.1439 - loss: 2.0754 - val_categorical_accuracy: 0.0500 - val_loss: 2.0963
Epoch 6/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - categorical_accuracy: 0.1531 - loss: 2.0748 - val_categorical_accuracy: 0.0500 - val_loss: 2.1001
Epoch 7/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - categorical_accuracy: 0.1867 - loss: 2.0645 - val_categorical_accuracy: 0.2000 - val_loss: 2.0670
Epoch 8/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - categorical_accuracy: 0.2268 - loss: 2.0316 - 

In [14]:
# Prediction
print("Starting prediction...")
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print("Prediction complete.")

Starting prediction...
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Prediction complete.


In [15]:
# Evaluation
accuracy = accuracy_score(y_true, y_pred_classes)
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')

Accuracy: 0.7
F1 Score: 0.6821284271284271


In [16]:
# Selecting a few samples (e.g., 5 samples) from the test set for prediction
num_samples_to_predict = 5
sample_indices = np.random.choice(X_test.shape[0], num_samples_to_predict, replace=False)  # Randomly pick indices without replacement
sample_data = X_test[sample_indices]
sample_labels = y_test[sample_indices]

In [17]:
# Make predictions
sample_predictions = model.predict(sample_data)
sample_pred_classes = np.argmax(sample_predictions, axis=1)
sample_true_classes = np.argmax(sample_labels, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


W0000 00:00:1717259596.348590     136 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [18]:
# Print the results
print("Sample predictions (class indices):", sample_pred_classes)
print("Actual labels (class indices):", sample_true_classes)

Sample predictions (class indices): [5 4 0 3 2]
Actual labels (class indices): [6 4 0 3 2]


In [19]:
# Map indices back to action names for clearer interpretation
predicted_actions = [actions[idx] for idx in sample_pred_classes]
actual_actions = [actions[idx] for idx in sample_true_classes]

In [20]:
# Selecting a few samples (e.g., 5 samples) from the test set for prediction
num_samples_to_predict = 8
sample_indices = np.random.choice(X_test.shape[0], num_samples_to_predict, replace=False)  # Randomly pick indices without replacement
sample_data = X_test[sample_indices]
sample_labels = y_test[sample_indices]

In [21]:
# Make predictions
sample_predictions = model.predict(sample_data)
sample_pred_classes = np.argmax(sample_predictions, axis=1)
sample_true_classes = np.argmax(sample_labels, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [22]:
# Print the results
print("Sample predictions (class indices):", sample_pred_classes)
print("Actual labels (class indices):", sample_true_classes)

Sample predictions (class indices): [0 6 2 3 5 0 2 0]
Actual labels (class indices): [0 6 7 7 5 0 4 0]


In [23]:
# Map indices back to action names for clearer interpretation
predicted_actions = [actions[idx] for idx in sample_pred_classes]
actual_actions = [actions[idx] for idx in sample_true_classes]

In [24]:
print("Predicted actions:", predicted_actions)
print("Actual actions:", actual_actions)

Predicted actions: ['MoveRight', 'FistClose', 'ThumbsUp', 'ThumbsDown', 'Circle', 'MoveRight', 'ThumbsUp', 'MoveRight']
Actual actions: ['MoveRight', 'FistClose', 'Victory', 'Victory', 'Circle', 'MoveRight', 'Stop', 'MoveRight']
